#### Cohesion:
- The degree to how elements in a certain class or function belong together
- Weak cohesion: function does a lot of different things that do not belong together
- Strong cohesion means that a function/class has a clear resposibility and performs only one task
- Strong cohesion makes code easier to maintain

#### Coupling:
- A measure on how dependent two parts of your code are on each other
- There will always be some coupling, but more coupling results in higher difficulty in mainting the code

### Before:

In [1]:
import string
import random

class VehicleRegistry:

    def generate_vehicle_id(self, length):
        return ''.join(random.choices(string.ascii_uppercase, k=length))

    def generate_vehicle_license(self, id):
        return f"{id[:2]}-{''.join(random.choices(string.digits, k=2))}-{''.join(random.choices(string.ascii_uppercase, k=2))}"


class Application:

    def register_vehicle(self, brand: string):
        # create a registry instance
        registry = VehicleRegistry()

        # generate a vehicle id of length 12
        vehicle_id = registry.generate_vehicle_id(12)

        # now generate a license plate for the vehicle
        # using the first two characters of the vehicle id
        license_plate = registry.generate_vehicle_license(vehicle_id)

        # compute the catalogue price
        # THIS PART OF THE CODE MAKES IT SO WE NEED A NEW ELIF STATEMENT
        # FOR EVERY CAR WE MAKE 
        catalogue_price = 0
        if brand == "Tesla Model 3":
            catalogue_price = 60000
        elif brand == "Volkswagen ID3":
            catalogue_price = 35000
        elif brand == "BMW 5":
            catalogue_price = 45000

        # compute the tax percentage (default 5% of the catalogue price, except for electric cars where it is 2%)
        # AND WE HAVE TO ADD THE MAKE OF THE CAR HERE IF THE CAR IS ELETRIC
        tax_percentage = 0.05
        if brand == "Tesla Model 3" or brand == "Volkswagen ID3":
            tax_percentage = 0.02

        # compute the payable tax
        payable_tax = tax_percentage * catalogue_price

        # print out the vehicle registration information
        print("Registration complete. Vehicle information:")
        print(f"Brand: {brand}")
        print(f"Id: {vehicle_id}")
        print(f"License plate: {license_plate}")
        print(f"Payable tax: {payable_tax}")

app = Application()
app.register_vehicle("Volkswagen ID3")

Registration complete. Vehicle information:
Brand: Volkswagen ID3
Id: RIYADFLSDRSD
License plate: RI-59-HT
Payable tax: 700.0


### After refactoring

In [8]:
class VehicleInfo:
    brand: str
    catalogue_price: int
    electric: bool
    
    def __init__(self, brand, electric, catalogue_price):
        self.brand = brand
        self.electric = electric
        self.catalogue_price = catalogue_price

    def compute_tax(self):
        tax_percentage = 0.05
        if self.electric:
            tax_percentage = 0.02
        return tax_percentage * self.catalogue_price

    def print(self):
        print(f"Brand: {self.brand}")
        print(f"Electric: {self.electric}")
        print(f"Payable tax: {self.compute_tax()}")

class Vehicle:
    id: str
    license_plate: str
    info: VehicleInfo

    def __init__(self, id, license_plate, info):
        self.id = id
        self.license_plate = license_plate
        # info is a VehicleInfo instance
        self.info = info
        print('self.info is an instance of VehicleInfo? ', isinstance(self.info, VehicleInfo))

    def print(self):
        print(f"Id: {self.id}")
        print(f"License plate: {self.license_plate}")
        # info is a VehicleInfo instance and that is why we can call its print method
        self.info.print()


class VehicleRegistry:

    def __init__(self):
        self.vehicle_info = { }
        # each of these will be a VehicleInfo instance stored in the self.vehicle_info dict
        self.add_vehicle_info("Tesla Model 3", True, 60000)
        self.add_vehicle_info("Volkswagen ID3", True, 35000)
        self.add_vehicle_info("BMW 5", False, 45000)
        self.add_vehicle_info("Tesla Model Y", True, 75000)

    def add_vehicle_info(self, brand, electric, catalogue_price):
        # VehicleInfo instance stored in the self.vehicle_info dict
        self.vehicle_info[brand] = VehicleInfo(brand, electric, catalogue_price)

    def generate_vehicle_id(self, length):
        # called by self.create_vehicle
        return ''.join(random.choices(string.ascii_uppercase, k=length))

    def generate_vehicle_license(self, id):
        # called by self.create_vehicle
        return f"{id[:2]}-{''.join(random.choices(string.digits, k=2))}-{''.join(random.choices(string.ascii_uppercase, k=2))}"

    def create_vehicle(self, brand):
        id = self.generate_vehicle_id(12)
        license_plate = self.generate_vehicle_license(id)
        return Vehicle(id, license_plate, self.vehicle_info[brand])


class Application:

    def register_vehicle(self, brand: string):
        # create a registry instance
        registry = VehicleRegistry()

        vehicle = registry.create_vehicle(brand)

        # print out the vehicle information
        vehicle.print()

app = Application()
app.register_vehicle("Volkswagen ID3")

self.info is an instance of VehicleInfo?  True
Id: KLOIDHRVPJOH
License plate: KL-01-KR
Brand: Volkswagen ID3
Electric: True
Payable tax: 700.0


1. Create instance of VehicleRegistry
    * \__init__ creates 4 key, value pairs where each value is an instance of VehicleBrand
2. Call create_vehicle
    * call self.generate_vechicle
    * call self.generate_vechicle_license
    * return an instance of Vehicle class
        * vehicle_info is an instance of VehicleInfo class